In [76]:
HTML("""
<iframe src='https://asaragga.github.io/GRF.jl/' width=1000 height=275></iframe>
    """)

HTML{String}("<iframe src='https://asaragga.github.io/GRF.jl/' width=1000 height=275></iframe>\n    ")

## 1. Instalar e Disponibilizar Aplicações Essenciais

(Carregar em >|Run para, sucessivamente, ir executando as células do notebook)

### 1.1 Instalar Aplicações Essenciais

Utilização remota, e.g. Binder: 

  * Mantêr todos os # 

Se fôr a primeira utilização local deste notebook :
  * Caso Julia Pro tenha sido instalada, apenas retirar os # na primeira e na segunda linha
  * Caso contrário, retirar # em todas as linhas (ou apenas naquelas relativas a aplicações ainda não instaladas)

In [ ]:
# import Pkg 
# Pkg.add("TimeSeries")
# Pkg.add("Distributions")      
# Pkg.add("StatsBase")          
# Pkg.add("HypothesisTests")    
# Pkg.add("StatsPlots")         
# Pkg.add("Interact")           

### 1.2 Disponibilizar Aplicações

In [ ]:
using LinearAlgebra, Distributions, StatsBase, HypothesisTests, StatsPlots, Interact, TimeSeries

## 2. Obter Dados

In [ ]:
VPortfolio = 74.6                                               
cambio = 0.884680                                               # Câmbio USD/EUR no final do dia 2019-04-15
dados = readtimearray("Dow-Jones.csv")                          # Tabela com retornos diários

Temos 33 colunas = ```:BT3m, :SP500, :MMM, :AXP, :AAPL, :BA, :CAT, :CVX, :CSCO, :KO, :DWDP, :XOM, :GS, :IBM, :INTC, :JNJ, :JPM,:MCD, :MRK, :MSFT, :NKE, :PFE, :PG, :HD, :TRV, :UTX, :UNH, :VZ, :V, :WBA, :WMT, :DIS, :USDEUR```

Por conveniência, iremos guardar os nomes das colunas no vetor ```simbolos(33x1)``` 

In [ ]:
simbolos = colnames(dados);   

Igualmente, iremos guardar as datas das observações no vetor imaginativamente chamado ```datas(1567x1)``` 

In [ ]:
datas = timestamp(dados);

## 3. Visualizar Dados

### 3.1 Séries Temporais de Retornos

In [ ]:
plot(dados[:SP500])         

In [ ]:
plot(plot(dados[:MMM]),plot(dados[:GS]),plot(dados[:MCD]),plot(dados[:DIS]))

In [ ]:
plot(dados[:USDEUR])        # Variação percentual diária da taxa de câmbio USD/EUR 

### 3.2 Histogramas

In [ ]:
R = values(dados)           # Transforma a Tabela dados(1567x33) na Matriz R(1567x33) equivalente 

  * ```R[:,j]``` seleciona todos os elementos da coluna ```j``` da matriz ```R```
  * ```R[i,:]``` seleciona todos os elementos da linha ```i``` da matriz ```R```

In [ ]:
plot(
histogram(R[:, 2], nbins=100, title = "S&P500", xlabel = "Retorno Diário S&P500", legend = false),
histogram(R[:,13], nbins=100, title = "Goldman-Sachs", xlabel = "Retorno Diário Goldman-Sachs", legend = false),
histogram(R[:,18], nbins=100, title = "McDonalds", xlabel = "Retorno Diário McDonalds", legend = false),
histogram(R[:,32], nbins=100, title = "Disney", xlabel = "Retorno Diário Walt Disney", legend = false))

### 3.3 Dependência Linear: Correlações

In [ ]:
plot(scatter(R[:,2],R[:, 3], title = "3M vs S&P500"          , markersize = 2, legend = false),
     scatter(R[:,2],R[:,13], title = "Goldman-Sachs vs S&P500", markersize = 2, legend = false),
     scatter(R[:,2],R[:,18], title = "McDonalds vs S&P500"    , markersize = 2, legend = false),
     scatter(R[:,2],R[:,32], title = "Walt Disney vs S&P500"  , markersize = 2, legend = false))

In [ ]:
println("correlação(3M,S&P500) = "           , cor(R[:, 3],R[:,2]))  
println("correlação(Goldman-Sachs,S&P500) = ", cor(R[:,13],R[:,2]))  
println("correlação(McDonalds,S&P500) = "    , cor(R[:,18],R[:,2]))  
println("correlação(Disney, S&P500) = "      , cor(R[:,32],R[:,2]))  

In [ ]:
scatter(R[:,2],R[:,33], legend=false, markersize=2, title="USD/EUR vs. S&P 500",xlabel="S&P500",ylabel="USD/EUR") 

In [ ]:
println("correlação(USD/EUR,S&P500)) = ", cor(R[:,33],R[:,2]))   

## 4. Composição do Portfólio

Definir matriz ```W(33x1)``` com os pesos no portfólio, sujeita às seguintes restrições:
  * ```0 <= w_i <= 1```      ausência de short-selling e financiamento do portfólio por empréstimos 
  * ```0 <= w_BT3m  <= 0.05```    peso máximo aplicado em BTs = 5%
  * ```0 <= w_SP500 <= 0.25```    peso máximo aplicado em instrumento replicador do S&P500 = 25%
  * Nº de açôes > 10, sendo o peso mínimo investido em cada >= 2.5%
  * ```sum(w_i) = 1```       todo o portfólio terá de ser aplicada em USD

In [ ]:
W = zeros(33)   # Preencher W com zeros inicialmente (32 oportunidades de investimento + câmbio USD/EUR)
W[1] = 0.0      #  0.0% investido em BTs (indice 1 de W)
W[2] = 0.1      # 10.0% num investimento replicando S&P500 (indice 2 de W)

for i = 3:32    # 90.0% remanescentes investidos em partes iguais pelas 30 ações do Down-Jones (indices 3-32 de W)
    W[i] = 0.03
end             # Notemos que W[33] = 0 

In [ ]:
sum(W)          # A soma dos pesos w_i deverá ser um valor muito próximo ou igual a 1

## 5. Simulação Histórica

### 5.1 Simular Historicamente os Retornos do Portfolio Atual

Simulação histórica dos retornos diários do portfolio: ```RP(1567x1) = R(1567x33) * W(33x1)```

In [ ]:
RP = R * W      

In [ ]:
histogram(RP, nbins=100, title = "Portfólio", xlabel = "Retorno Diário", legend = false)

### 5.2 Cálculo do VaR: Abordagem Não-Paramétrica ($\mathrm{np}$)

$$\mathrm{VaR} = Z_\alpha V_p$$ onde $$Z_\alpha$$ é o $\alpha$-quantil da **Distribuição Empírica** dos retornos obtidos por simulação histórica. A função ```quantile(X,alfa)``` calcula o ```alfa```-quantil de ```X```.

In [ ]:
println("DEaR(5%) simulação histórica não paramétrica = ", - quantile(RP,0.05) * VPortfolio, " USD 1x10^6")
println("DEaR(1%) simulação histórica não paramétrica = ", - quantile(RP,0.01) * VPortfolio, " USD 1x10^6")

### 5.3 Cálculo do VaR: Abordagem Semi-Paramétrica ($\mathrm{sp}$)

$$\text{VaR}_\alpha = - Z_\alpha V_p$$ com $$Z_\alpha = z_\alpha \sigma_p $$ onde o $\alpha$-quantil da distribuição empírica $Z_\alpha$ é agora determinado pelo produto de dois componemtes: ($\textrm{i}$) quantil $z_\alpha$ da distribuição padronizada escolhida, por exemplo $z_{5\%}$ será igual a -1.645 se a distribuição padronizada for a $\mathcal{N}(0,1)$ e ($\textrm{ii}$) $\sigma_p$ determinado por simulação histórica dos retornos do portfólio.

Iremos pois tentar encontrar em (5.3.1) uma distribuição analítica (e.g ```Normal```, ```T-Student```) que se ajuste bem à distribuição empírica dos retornos diários do portfólio já calculados em (5.1) por simulação histórica.

#### 5.3.1 Escolha da Distribuição Analítica

Nota para os cálculos que se seguem: o 'ponto'```.``` aplica um operador (e.g. +, *, /, exp, log, sin,...) a todos os elementos de um vetor/matriz. 

Exemplo: sendo ```x = [1.0,2.0,3.0]```, tem-se:
  * ```x.^2 = [1.0^2, 2.0^2, 3.0^2]```
  * ```log.(x .+ 1.0) = [log(1.0)+1.0,log(2.0)+1.0,log(3.0)+1.0]```

De referir ainda que, como é habitual em linguagens de programação, o ln(x) é calculado através da função ```log(x)```

In [ ]:
mu = mean(RP)                 # média dos retornos do portfólio
sigma = std(RP)               # desvio-padrão dos retornos do portfólio
RP01 = (RP .- mu)./sigma;     # retornos do portfólio padronizados, com média = 0 e desvio-padrão = 1         

In [ ]:
histogram(RP01, nbins=100, normed=true, label="Empírica Padronizada")
plot!(Normal(0,1), label = "Normal(0,1)")
plot!(TDist(6), label = "t-Student(6)")

In [ ]:
plot(TDist(6), label = "t-Student(6)")
plot!(Normal(0,1), label = "Normal(0,1)")

Conseguimos ter uma melhor visualização do que se passa nas caudas das distribuições através de um gráfico quantil-quantil (QQ). A função ```qqplot(Distribuição,X)```permite essa análise. Escolhemos analisar as distribuições Normal(0,1) e T-Student(g) com g = 5, 6, 7, 8, 9.

In [ ]:
plot(                             
qqplot(Normal,   RP01, markersize=1, title = "Normal"),     # Normal(0,1)  vs. distribuição empírica padronizada
qqplot(TDist(5), RP01, markersize=1, title = "TDist(5)"),   # T-Student(5) vs. distribuição empírica padronizada
qqplot(TDist(6), RP01, markersize=1, title = "TDist(6)"),           
qqplot(TDist(7), RP01, markersize=1, title = "TDist(7)"),
qqplot(TDist(8), RP01, markersize=1, title = "TDist(8)"),
qqplot(TDist(9), RP01, markersize=1, title = "TDist(9)")
)

In [ ]:
@manipulate for obs = 1:10
    qqplot(TDist(obs), RP01,
    markersize = 2, 
    title = "Distribuição Emprírica vs. T-Student com $(obs) graus liberdade",
    legend = :bottomright,
    xlabel = "Distribuição T-Student($(obs))", 
    ylabel = "Distribuição Empírica",
    label = "Observações")
end    
# MOVER o cursor (obs) no topo do gráfico seguinte para variar os graus de liberdade da distribuição T-Student!

#### 5.3.2 Cálculo do VaR a 5% e 1%

In [ ]:
println("DEaR(5%) simulação histórica (sp) = ", - quantile(TDist(7),0.05) * sigma * VPortfolio , " USD 1x10^6")
println("DEaR(1%) simulação histórica (sp) = ", - quantile(TDist(7),0.01) * sigma * VPortfolio , " USD 1x10^6")

## 6. Análise de Stress 

Iremos analisar os cenários de stress correspondentes às (1%) piores perdas do portfolio. Depois de criarmos uma nova tabela ```TabelaRP``` apenas com os retornos do portfolio e respetivas datas, iremos identificar as datas em que as piores perdas ocorreram.

In [ ]:
TabelaRP = TimeArray(datas,RP,[:RP]); # Tabela temporal com os retornos simulados do portfólio (coluna :RP) e datas

O passo seguinte será pois determinar para o portfólio quais os dias em que se verificaram retornos simulados $\leq$ 1%-quantil

In [ ]:
stress = TabelaRP[ findall(TabelaRP[:RP] .<= quantile(RP,0.01)) ]  

Certamente o ano de 2018 teria sido bastante stressante para o portfólio. O pior dia foi 5 Fevereiro de 2018, com uma queda de -4.44%. Vamos tentar ver o que poderá ter acontecido de especial nesse dia, quer nos mercados NYSE e NASDAQ, quer nos próprios EUA.

Em 5 Fevereiro 2018, o indíce Dow caiu -4.6%: [Dow suffers worst one-day point fall in history](https://www.youtube.com/watch?v=GJMmK0sAYBk). Mas porque é que os mercados abruptamente cairam tanto nesse dia? Frequentemente não é possivel associar uma causa económica ou política direta aos eventos extremos de 'crash' nas bolsas de ações nos dias precisos em que ocorrem. 

Tal não será porém o caso do dia 10 Outubro 2018, em que se verificou uma queda de -2.99% no valor do portfolio: [World stock markets dive as Trump attacks 'crazy' US rate hikes](https://www.theguardian.com/business/2018/oct/11/asian-stock-markets-dive-as-trump-attacks-crazy-us-rate-hikes).

## 7. Análise de Desempenho do Portfólio

Vamos analisar o desempenho do portfolio face ao desempenho do benchmark escolhido. Medidas de desempenho têm de levar em conta não só o retorno obtido, mas também o risco associado ao investimento no portfólio. Uma medida muito utilizada é o rácio Sharpe, criado por William Sharpe um dos autores do CAPM. O rácio Sharpe ex-ante é definido da seguinte forma:


$$S = \frac{E[R-R_b]}{\sqrt{\mathrm{var}[R-R_b]}}$$

onde $R$ é o retorno do portfólio e $R_b$ é o retorno do seu benchmark. Estamos contudo mais interessados na sua concretização empírica, o rácio Sharpe ex-post, que utiliza os retornos realizados e não o valor esperado de retornos futuros.
Denotando os retornos em excesso do portfólio face ao benchmark (os quais podem ser negativos) por $X_i$,

$$X_i = R_i-R_{bi}$$
$$\mu_x=\tfrac{1}{N}\sum_{i=1}^N X_i$$
e
$$S = \frac{\mu_x}{\sqrt{\tfrac{1}{N}\sum_{i=1}^N (X_i-\mu_x)^2}}$$
onde normalmente se utiliza a variância da amostra.

### 8.1 Portfolio vs. S&P500

O benchmark natural do portfólio será o índice S&P500:

In [ ]:
X = RP .- R[:,2]                    # Vetor com os retornos em excesso do portfólio face ao S&P 500
Sharpe = mean(X)/std(X)             # Notemos que mean(X) e std(X) são números e não vetores, daí não usarmos ./

### 8.2 Portfolio vs. BT3m

Utilizando agora aplicações em Bilhetes do Tesouro a 3 meses como benchmark do portfólio:

In [ ]:
X_P = RP .- R[:,1]                  # Vetor com os retornos em excesso do portfólio face a BT3m
Sharpe = mean(X_P)/std(X_P)